In [ ]:
 ## !pip install ultralytics norfair --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 ## !pip install -U numpy pandas --force-reinstall --no-cache-dir

In [ ]:
import os
import cv2
import torch
import numpy as np
from ultralytics import YOLO
from norfair import Detection, Tracker
import random
import platform
import subprocess
import pandas as pd
from tqdm import tqdm
from google.colab import files

In [ ]:
USE_VIDEO_UPLOAD = True
VIDEO_FPS = 15
YOLO_MODEL = "yolov8n.pt"
INPUT_FOLDER = "frames"
OUTPUT_FOLDER = "output"
os.makedirs(INPUT_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
if USE_VIDEO_UPLOAD:
    print("[INFO] Please upload your video file (MP4/AVI/etc)...")
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0]

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_name = f"{frame_count:05}.jpg"
        cv2.imwrite(os.path.join(INPUT_FOLDER, frame_name), frame)
        frame_count += 1
    cap.release()
    print(f"[INFO] Extracted {frame_count} frames to {INPUT_FOLDER}/")

[INFO] Please upload your video file (MP4/AVI/etc)...


Saving 15sec_input_720p.mp4 to 15sec_input_720p.mp4
[INFO] Extracted 375 frames to frames/


In [ ]:
model = YOLO(YOLO_MODEL)
tracker = Tracker(distance_function=lambda d, t: np.linalg.norm(d.points - t.estimate), distance_threshold=30)

100%|██████████| 6.25M/6.25M [00:00<00:00, 122MB/s]


In [ ]:
id_colors = {}
def get_color_for_id(player_id):
    if player_id not in id_colors:
        random.seed(player_id)
        id_colors[player_id] = tuple(random.randint(0, 255) for _ in range(3))
    return id_colors[player_id]

In [ ]:
frame_files = sorted(os.listdir(INPUT_FOLDER))
fps = VIDEO_FPS
tracking_results = []

In [ ]:
for i, fname in enumerate(tqdm(frame_files, desc="Processing frames")):
    frame_path = os.path.join(INPUT_FOLDER, fname)
    frame = cv2.imread(frame_path)

    results = model(frame)
    bboxes = results[0].boxes.xyxy.cpu().numpy()

    detections = []
    centers = []

    for box in bboxes:
        x1, y1, x2, y2 = map(int, box)
        cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
        centers.append((x1, y1, x2, y2, cx, cy))
        detections.append(Detection(points=np.array([[cx, cy]])))

    tracked_objects = tracker.update(detections)

    for obj in tracked_objects:
        est = obj.estimate[0]
        ID = obj.id
        color = get_color_for_id(ID)

        matched_bbox = None
        for (x1, y1, x2, y2, cx, cy) in centers:
            if abs(est[0] - cx) < 20 and abs(est[1] - cy) < 20:
                matched_bbox = (x1, y1, x2, y2)
                break

        if matched_bbox:
            x1, y1, x2, y2 = matched_bbox
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        cv2.circle(frame, (int(est[0]), int(est[1])), 6, color, -1)
        cv2.putText(frame, f"ID {ID}", (int(est[0]), int(est[1]) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        tracking_results.append({'frame': i, 'id': ID, 'cx': est[0], 'cy': est[1]})

    seconds = i // fps
    timer_text = f"{seconds // 60:02}:{seconds % 60:02}"
    cv2.putText(frame, timer_text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                1.2, (255, 255, 255), 3, cv2.LINE_AA)

    cv2.imwrite(os.path.join(OUTPUT_FOLDER, fname), frame)

print("[INFO] Player re-identification tracking completed.")

Processing frames:   0%|          | 0/375 [00:00<?, ?it/s]


0: 384x640 21 persons, 476.0ms
Speed: 16.6ms preprocess, 476.0ms inference, 34.3ms postprocess per image at shape (1, 3, 384, 640)


Processing frames:   0%|          | 1/375 [00:08<56:02,  8.99s/it]


0: 384x640 18 persons, 1 sports ball, 247.7ms
Speed: 4.1ms preprocess, 247.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)


Processing frames:   1%|          | 2/375 [00:09<24:03,  3.87s/it]

...


0: 384x640 4 persons, 142.0ms
Speed: 2.9ms preprocess, 142.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing frames: 100%|█████████▉| 374/375 [01:24<00:00,  5.74it/s]


0: 384x640 4 persons, 1 tennis racket, 138.9ms
Speed: 2.6ms preprocess, 138.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing frames: 100%|██████████| 375/375 [01:25<00:00,  4.41it/s]

[INFO] Player re-identification tracking completed.


In [ ]:
pd.DataFrame(tracking_results).to_csv("tracking_log.csv", index=False)
print("[INFO] Tracking log saved to tracking_log.csv")

[INFO] Tracking log saved to tracking_log.csv


In [ ]:
output_video_path = "player_tracking_output.mp4"
frame_example = cv2.imread(os.path.join(OUTPUT_FOLDER, frame_files[0]))
height, width, _ = frame_example.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

print("[INFO] Saving output video...")

[INFO] Saving output video...


In [ ]:
for fname in frame_files:
    frame = cv2.imread(os.path.join(OUTPUT_FOLDER, fname))
    video_writer.write(frame)

video_writer.release()
print(f"[INFO] Output video saved as {output_video_path}")

[INFO] Output video saved as player_tracking_output.mp4


In [ ]:
files.download("tracking_log.csv")
files.download(output_video_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>